In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

import keras
from keras import layers, ops

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os, sys

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current sessioㅔ갸ㅜ

print('loaded.')

2024-06-29 16:21:38.190161: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-29 16:21:38.190271: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-29 16:21:38.326999: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


loaded.


In [2]:
class ContextTargetExtractor(layers.Layer):
    def __init__(self, patch_size=32, context_scale=(0.5, 0.75), 
                 target_scale=(0.15, 0.75), target_aspect_ratio=(0.75, 1.5), num_targets=4, padding=False, **kwargs):
        super(ContextTargetExtractor, self).__init__(**kwargs)
        self.patch_size = patch_size
        self.context_scale = context_scale
        self.target_scale = target_scale
        self.target_aspect_ratio = target_aspect_ratio
        self.num_targets = num_targets
        self.padding = padding

    def call(self, inputs):
        batch_size, img_height, img_width, channels = ops.shape(inputs)[0], ops.shape(inputs)[1], ops.shape(inputs)[2], ops.shape(inputs)[3]

        # 이미지를 패치로 나누기
        patches = tf.image.extract_patches(images=inputs,
                                           sizes=[1, self.patch_size, self.patch_size, 1],
                                           strides=[1, self.patch_size, self.patch_size, 1],
                                           rates=[1, 1, 1, 1],
                                           padding='VALID')
        num_patches_x = img_width // self.patch_size
        num_patches_y = img_height // self.patch_size
        num_patches = num_patches_x * num_patches_y
        patches = ops.reshape(patches, [batch_size, num_patches_y, num_patches_x, self.patch_size, self.patch_size, channels])

        # Context와 Target 블록 생성
        context_block, context_positions = self.sample_block(patches, self.context_scale, img_height, img_width, num_patches_x, num_patches_y)
        target_blocks = []
        target_positions = []
        for _ in range(self.num_targets):
            target_block, target_position = self.sample_block(patches, self.target_scale, img_height, img_width, num_patches_x, num_patches_y, context_positions)
            target_blocks.append(target_block)
            target_positions.append(target_position)
        
        # Context 블록을 원래 이미지 크기로 zero padding (필요할 경우)
        if self.padding:
            context_block = tf.image.pad_to_bounding_box(context_block, 0, 0, img_height, img_width)
        
        return context_block, target_blocks, context_positions, target_positions

    def sample_block(self, patches, scale_range, img_height, img_width, num_patches_x, num_patches_y, context_positions=None):
        scale = np.random.uniform(scale_range[0], scale_range[1])
        block_num_patches_y = max(1, int(scale * num_patches_y))
        block_num_patches_x = max(1, int(block_num_patches_y * np.random.uniform(self.target_aspect_ratio[0], self.target_aspect_ratio[1])))
        
        block_num_patches_x = min(block_num_patches_x, num_patches_x)  # Ensure block fits within image
        block_num_patches_y = min(block_num_patches_y, num_patches_y)  # Ensure block fits within image

        for _ in range(100):
            start_patch_y = np.random.randint(0, num_patches_y - block_num_patches_y + 1)
            start_patch_x = np.random.randint(0, num_patches_x - block_num_patches_x + 1)
            end_patch_y = start_patch_y + block_num_patches_y
            end_patch_x = start_patch_x + block_num_patches_x

            block_positions = [(y * num_patches_x + x) for y in range(start_patch_y, end_patch_y) for x in range(start_patch_x, end_patch_x)]
            
            if context_positions is None or not any(pos in context_positions for pos in block_positions):
                break

        block = patches[:, start_patch_y:end_patch_y, start_patch_x:end_patch_x, :, :, :]
        block = ops.reshape(block, [ops.shape(patches)[0], block_num_patches_y * self.patch_size, block_num_patches_x * self.patch_size, ops.shape(patches)[-1]])
        
        return block, block_positions
# 임의의 이미지 데이터를 사용한 예제
image = tf.random.normal((32, 256, 256, 3))

# Context와 Target 추출 레이어
extractor = ContextTargetExtractor(patch_size=32, padding=False)
context, targets, context_positions, target_positions = extractor(image)

context_shape = context.shape
context_positions_list = context_positions
target_shapes = [target.shape for target in targets]
target_positions_list = target_positions

context_shape, context_positions_list, target_shapes, target_positions_list

(TensorShape([32, 160, 160, 3]),
 [26,
  27,
  28,
  29,
  30,
  34,
  35,
  36,
  37,
  38,
  42,
  43,
  44,
  45,
  46,
  50,
  51,
  52,
  53,
  54,
  58,
  59,
  60,
  61,
  62],
 [TensorShape([32, 32, 32, 3]),
  TensorShape([32, 64, 64, 3]),
  TensorShape([32, 96, 128, 3]),
  TensorShape([32, 32, 32, 3])],
 [[55], [12, 13, 20, 21], [2, 3, 4, 5, 10, 11, 12, 13, 18, 19, 20, 21], [17]])